In [139]:
plot_graph(proteins = ["Tenascin"], diseases = ["Drug toxicity", "Раны и травмы"], depth = 1)

{'data': [],
 'directed': False,
 'multigraph': False,
 'elements': {'nodes': [{'data': {'id': 'Tenascin',
     'value': 'Tenascin',
     'name': 'Tenascin'}},
   {'data': {'id': 'Lung Injury',
     'value': 'Lung Injury',
     'name': 'Lung Injury'}},
   {'data': {'id': 'Chronic Lung Injury',
     'value': 'Chronic Lung Injury',
     'name': 'Chronic Lung Injury'}}],
  'edges': [{'data': {'source': 'Tenascin', 'target': 'Lung Injury'}},
   {'data': {'source': 'Tenascin', 'target': 'Chronic Lung Injury'}}]}}

In [1]:
# загрузка необходимых таблиц

import pandas as pd

tb2 = pd.read_csv("tb2.tsv", sep = "\t")
#edges_df = pd.read_csv("edges_df.tsv", sep = "\t")
dis = pd.read_csv("diseases.tsv", sep = "\t")
#nodes_df = pd.read_csv("C:/Users/a120/MRM/nodes_df.tsv", sep = "\t")

In [2]:
def concat(*args):
    return pd.concat([x for x in args if not x.empty])

In [62]:
# загрузка необходимых пакетов

import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from io import StringIO

In [138]:
def plot_graph(table = tb2, proteins = None, diseases = None, depth = 0):
    # функция строит граф ассоциации болезнь-белок и записывает граф в текстовый файл GEFX
    # на вход подается таблица, список белков и список болезней
    # список белков и список болезней - в формате списка []
    # аргумент depth показывает, до какой "глубины" нужно рисовать граф:
    # только про классы или про болезни вообще
    # depth = 0 --> только про классы (если изначально как diseases не введена болезнь)
    # depth = 1 --> про болезни
    
    diseases_list = []
    classes_list = []
    table1 = pd.DataFrame()
    table2 = pd.DataFrame()
    
    if diseases != None:
        for item in diseases:
            if item in list(dis.diseaseName):
                diseases_list.append(item)
            if item in list(dis.disease):
                classes_list.append(item)
    
    if depth == 0:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
            
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
            
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф    
        G = networkx.from_pandas_edgelist(table, "Protein", "disease")
            
    elif depth == 1:
        if proteins != None:
            proteins = pd.DataFrame(proteins)
            proteins.columns = ["Protein"]
            table = table.merge(proteins, on = "Protein")
        if len(diseases_list) != 0:
            diseases_list = pd.DataFrame(diseases_list)
            diseases_list.columns = ["diseaseName"]
            table1 = table.merge(diseases_list, on = "diseaseName")
        if len(classes_list) != 0:
            classes_list = pd.DataFrame(classes_list)
            classes_list.columns = ["disease"]
            table2 = table.merge(classes_list, on = "disease")
        if not table1.empty or not table2.empty:
            table = concat(*[table1, table2])
            table = table.drop_duplicates()
        # собственно, создаем граф
        G = networkx.from_pandas_edgelist(table, "Protein", "diseaseName")

    return networkx.cytoscape_data(G)